# Computing customer-centric metrics of managerial interest

Data Konferences - Madrid, 8th February 2018

In [1]:
"""
Implementation of the beta-geometric/NBD (BG/NBD) model from '"Counting Your Customers" the Easy Way: An Alternative to
the Pareto/NBD Model' (Fader, Hardie and Lee 2005) http://brucehardie.com/papers/018/fader_et_al_mksc_05.pdf and
accompanying technical note http://www.brucehardie.com/notes/004/

Apache 2 License
"""
from math import log, exp

import numpy as np
from scipy.optimize import minimize

from scipy.special import gammaln

import pandas as pd

def log_likelihood_individual(r, alpha, a, b, x, tx, t):
    """Log of the likelihood function for a given randomly chosen individual with purchase history = (x, tx, t) where
    x is the number of transactions in time period (0, t] and tx (0 < tx <= t) is the time of the last transaction"""
    ln_a1 = gammaln(r + x) - gammaln(r) + r * log(alpha)
    ln_a2 = gammaln(a + b) + gammaln(b + x) - gammaln(b) - gammaln(a + b + x)
    ln_a3 = -(r + x) * log(alpha + t)
    a4 = 0
    if x > 0:
        a4 = exp(log(a) - log(b + x - 1) - (r + x) * log(alpha + tx))
    return ln_a1 + ln_a2 + log(exp(ln_a3) + a4)


def negative_log_likelihood(r, alpha, a, b, customers):
    """Sum of the individual log likelihoods"""
    # can't put constraints on n-m minimizer so fake them here
    if r <= 0 or alpha <= 0 or a <= 0 or b <= 0:
        return -np.inf
    return sum([log_likelihood_individual(r, alpha, a, b, x, tx, t) for x, tx, t in customers])


def maximize(customers):
    negative_ll = lambda params: -negative_log_likelihood(*params, customers=customers)
    params0 = np.array([1., 1., 1., 1.])
    res = minimize(negative_ll, params0, method='nelder-mead', options={'xtol': 1e-8})
    return res


def fit(customers):
    res = maximize(customers)
    if res.status != 0:
        raise Exception(res.message)
    return res.x


def load_data(fname):
    data = []
    with open(fname) as f:
        f.readline()
        for line in f:
            data.append(map(float, line.strip().split(',')[1:4]))
    return data


def prob_alive(r, alpha, a, b, x, t_x, T): 
    """Computes the probability of being alive at T given r, alpha, a, b, frequency, recency and T"""
    return 1/(1 + x * ((a/(b + x - 1)) * ((alpha + T)/(alpha + t_x))**(r + x)))


def model_data(data):
    r, alpha, a, b = fit(data)
    print ("r: {0}, alpha: {1}, a: {2}, b: {3}".format(r, alpha, a, b))
    return {'r':r, 'alpha':alpha ,'a':a, 'b':b }

# Data
---

In [2]:
# Source: http://www.brucehardie.com/notes/026/
data = load_data('customers.csv')
df = pd.DataFrame(data, columns = ["frequency", "recency", "T"])
df.head()

,frequency,recency,T
0,2.0,30.43,38.86
1,1.0,1.71,38.86
2,0.0,0.00,38.86
3,0.0,0.00,38.86
4,0.0,0.00,38.86


In [3]:
print ("Número de clientes: {0}"
       .format(df.shape[0]))

Número de clientes: 2357


### Average customer purchases

In [4]:
print ("Compras media por cliente: {0}"
       .format(df[df['frequency'] > 0]['frequency'].median()))

Compras media por cliente: 2.0


### Repeat customer rate

In [5]:
print ("Porcentaje de clientes con más de una compra: {0:.0f}%."
       .format(round(sum(df['recency'] != 0)/float(len(df)) * 100,2 )))

Porcentaje de clientes con más de una compra: 40%.


# Model
---

In [6]:
model_params = model_data(data)

#Extract model params
r = model_params['r']
alpha = model_params['alpha']
a = model_params['a']
b = model_params['b']

r: 0.242592954211, alpha: 4.41353190403, a: 0.792886814457, b: 2.4257558827


### Purchase process
$r$ and alpha $\alpha$ describe the gamma mixing distribution of the NBD transaction process

In [7]:
_lambda = r/alpha
print ("Frecuencia de compra media: 1 compra cada {0} semanas"
       .format(round(1/_lambda,5 )))

Frecuencia de compra media: 1 compra cada 18.19316 semanas


### Dropout Process
$a$ and $b$ describe the beta mixing distribution of the beta geometric
dropout probabilities.

In [8]:
p = a/(a + b)
print ("Probabilidad media de abandono tras la compra: {0:.2f}"
       .format(p))

Probabilidad media de abandono tras la compra: 0.25


### Lifetime

In [9]:
p_alive = 1 / (_lambda * p)
print ("Los clientes compran durante {0:.0f} semanas"
       .format(p_alive))

Los clientes compran durante 74 semanas


### Probability that a customer with purchase history (x, tx, T) is “alive” at time T

In [10]:
#P(Alive) of a customer who has the same recency and total time observed.
prob_alive(r, alpha, a, b, 1, 30, 39 )
print ("Probabilidad de vida de un cliente que cuya última compra ha sido hace 9 semanas: {0}"
       .format(round(prob_alive(r, alpha, a, b, 1, 30, 39 ),4)))

Probabilidad de vida de un cliente que cuya última compra ha sido hace 9 semanas: 0.6963


In [11]:
print ("Probabilidad de vida de un cliente que ha comprado en la última semana: {0}"
       .format(round(prob_alive(r, alpha, a, b, 1, 39, 39 ),4)))

Probabilidad de vida de un cliente que ha comprado en la última semana: 0.7537
